In [10]:
%matplotlib inline
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [13]:
data = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=30min&apikey=T694IP72UNMJTRME&datatype=csv')
df = pd.read_csv('intraday_30min_MSFT.csv')

In [14]:
df.head(20)

,timestamp,open,high,low,close,volume
0,2019-05-03 16:00:00,128.8000,129.0300,128.7400,128.8900,2387512
1,2019-05-03 15:30:00,128.7600,128.8300,128.5800,128.8000,1943139
2,2019-05-03 15:00:00,129.0047,129.0500,128.7100,128.7600,1380406
3,2019-05-03 14:30:00,129.2900,129.3100,128.8610,129.0000,1048848
4,2019-05-03 14:00:00,129.1400,129.4200,129.0966,129.2900,1898223
5,2019-05-03 13:30:00,128.6200,129.1450,128.5500,129.1390,1592331
6,2019-05-03 13:00:00,128.6400,128.7500,128.3800,128.6192,1176682
7,2019-05-03 12:30:00,128.3400,128.6400,128.2756,128.6400,1309346
8,2019-05-03 12:00:00,128.5700,128.6150,128.1600,128.3500,1213747
9,2019-05-03 11:30:00,128.4950,128.7725,128.4601,128.5750,1835496
